# https://github.com/snntorch/Spiking-Neural-Networks-Tutorials/blob/main/tutorial_1_spikegen.ipynb

![image.png](attachment:image.png)

![image.png](attachment:image.png)

In [20]:
# !pip install snntorch

In [21]:
#!pip install deepinsight

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
from matplotlib import rcParams
import matplotlib.gridspec as gridspec
from sklearn import linear_model
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torch.utils.data as data
import torch.optim as optim
import torchvision.datasets as datasets
from torchsummary import summary
#import deepinsight

import os

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# This is for Downloading the data

In [24]:
import requests # for downloading using urls
os.chdir("/content/drive/MyDrive/kaveh_dataset")
# Initialize an empty list to store filenames
fname = []

# Loop through the range [0, 3) to generate filenames and store them in the list
for j in range(3):
    # j is replaced in the %d
    fname.append('steinmetz_part%d.npz' % j)

# List of URLs for data download
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

# Loop through the list of URLs and download corresponding files
for j in range(len(url)): # j starts at 0
    # Check if the file does not exist before attempting download
    if not os.path.isfile(fname[j]):
        try:
            # Send a GET request to download the file
            r = requests.get(url[j])
        except requests.ConnectionError:
            # If there is a connection error, print a failure message
            print("!!! Failed to download data !!!")
        # Check if the request was successful (status code 200)
        if r.status_code != requests.codes.ok:
            # If the download request was not successful, print a failure message
            print("!!! Failed to download data !!!")
        else:
            # If the download was successful, write the content to the corresponding file
            with open(fname[j], "wb") as fid:
                fid.write(r.content)

# Loading Data and showing it for insight

In [25]:
# Set some configurations for the figure size and axis spines of plots
rcParams['figure.figsize'] = [16, 10]   # Set the size of the figure to 16 inches in width and 10 inches in height
rcParams['axes.spines.top'] = False     # Turn off the top axis spine (border line)
rcParams['axes.spines.right'] = False   # Turn off the right axis spine

# Enable figure autolayout, which adjusts the subplot parameters to fit the figure area automatically
rcParams['figure.autolayout'] = True

# Groupings of brain regions
regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
brain_groups = [  # A list of lists representing different groups of brain regions
    ["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"],     # visual cortex
    ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"],   # thalamus
    ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"],      # hippocampal
    ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP", "TT"],  # non-visual cortex
    ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"],   # midbrain
    ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"],   # basal ganglia
    ["BLA", "BMA", "EP", "EPd", "MEA"]    # cortical subplate
]

# Create a copy of brain_groups to store all brain regions (for later use)
brain_groups_all = brain_groups[:]
n_regions = len(regions)   # Get the total number of brain regions

# Load the data
def load_alldat():
    alldat = np.array([])   # Initialize an empty numpy array to store data
    for j in range(len(fname)):    # Loop through the range of available files
        # Load data from the corresponding file and concatenate it to alldat
        alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz' % j, allow_pickle=True)['dat']))
    return alldat   # Return the concatenated data

alldat = load_alldat()   # Load all the data from the files into alldat
dt = alldat[0]['bin_size']   # Get the bin size from the first element of the data

In [26]:
alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx.
dat = alldat[11]
print(dat.keys())

dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])


In [27]:
spikes_all = dat['spks']

In [28]:
print(spikes_all.shape)

(698, 340, 250)


In [29]:
spdat = []
for i in range(len(alldat)):
    sp = np.array(alldat[i]['spks'][:520,:,:])
    print(sp.shape)
    spdat.append(sp)
    print(len(spdat))

(520, 214, 250)
1
(520, 251, 250)
2
(520, 228, 250)
3
(520, 249, 250)
4
(520, 254, 250)
5
(520, 290, 250)
6
(520, 252, 250)
7
(520, 250, 250)
8
(520, 372, 250)
9
(520, 447, 250)
10
(520, 342, 250)
11
(520, 340, 250)
12
(520, 300, 250)
13
(520, 268, 250)
14
(520, 404, 250)
15
(474, 280, 250)
16
(520, 224, 250)
17
(520, 316, 250)
18
(520, 247, 250)
19
(520, 235, 250)
20
(520, 124, 250)
21
(520, 444, 250)
22
(520, 151, 250)
23
(520, 187, 250)
24
(520, 261, 250)
25
(520, 178, 250)
26
(520, 253, 250)
27
(520, 142, 250)
28
(520, 128, 250)
29
(520, 143, 250)
30
(520, 237, 250)
31
(520, 260, 250)
32
(520, 191, 250)
33
(520, 296, 250)
34
(520, 311, 250)
35
(520, 258, 250)
36
(520, 181, 250)
37
(520, 199, 250)
38
(520, 343, 250)
39


In [30]:
spikesList = np.array([item for items in spdat for item in items])

<ipython-input-30-931c5c169877>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  spikesList = np.array([item for items in spdat for item in items])


In [12]:
print(spikesList.shape)
spikesList[9].shape

(20234,)


(214, 250)

In [52]:
feed = []
for session in range(len(spdat)):
    print(session)
#     print(session.shape)
    dat = spdat[session]
    fb = alldat[session]['feedback_type']
    #print(dat.shape)
    spikes_all = np.swapaxes(dat,0,1)
    #print(spikes_all.shape)
    spikes_all = np.nan_to_num(spikes_all.reshape(spikes_all.shape[0],-1))
    #spikes_all = pd.Series(spikes_all).fillna(0).tolist()
    print(spikes_all.shape)
    feed.append(fb)
    try:
      if session == 0:
        spikes = spikes_all
      else:
        spikes = np.vstack((spikes,spikes_all))
    except:
      pass
    #spikes = spikes.append(spike)
    #spike = spike.append(spikes_all)

0
(214, 130000)
1
(251, 130000)
2
(228, 130000)
3
(249, 130000)
4
(254, 130000)
5
(290, 130000)
6
(252, 130000)
7
(250, 130000)
8
(372, 130000)
9
(447, 130000)
10
(342, 130000)
11
(340, 130000)
12
(300, 130000)
13
(268, 130000)
14
(404, 130000)
15
(280, 118500)
16
(224, 130000)
17
(316, 130000)
18
(247, 130000)
19
(235, 130000)
20
(124, 130000)
21
(444, 130000)
22
(151, 130000)
23
(187, 130000)
24
(261, 130000)
25
(178, 130000)
26
(253, 130000)
27
(142, 130000)
28
(128, 130000)
29
(143, 130000)
30
(237, 130000)
31
(260, 130000)
32
(191, 130000)
33
(296, 130000)
34
(311, 130000)
35
(258, 130000)
36
(181, 130000)
37
(199, 130000)
38
(343, 130000)


In [80]:
feed[0].shape

(214,)

In [77]:
spikes.shape

(9770, 130000)

In [54]:
#---------------------------------------------------

In [ ]:
len(alldat) # number of samples in the fisrt container


39

In [ ]:
len(alldat[0]) # Access to the first memebr of the subcontainer (second hierarcgy)

31

In [ ]:
alldat[1].keys() # name of classes

dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])

In [ ]:
len(alldat[1].keys()) # number of classes

31

In [ ]:
# number of pic and resolution in each sub-container
i = 0
for session in alldat:# session = sub-container
    print(i)
    print(session["spks"].shape)
    i+=1

In [ ]:
len(alldat)

39

In [5]:
# set the default configuration settings for Matplotlib
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['axes.prop_cycle'] = plt.cycler('color', ['r', 'g', 'b', 'y'])

In [6]:
all_spike_data = list(np.swapaxes(session["spks"],0,1).reshape(np.swapaxes(session["spks"],0,1).shape[0],-1) for session in alldat)

In [7]:
all_spike_data[0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [9]:
all_spike_data[0][0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [ ]:
len(all_spike_data[0])

214

In [55]:
def maping(arr):
    for i in range(len(arr)):
        if arr[i] == 1:
            arr[i] =1
        else:
            arr[i] = 0
    return arr

In [ ]:
#all_Y = [1 if Y == 1 else 0 for Y in session["feedback_type"] for session in alldat]

In [83]:
all_Y = [maping(fe) for fe in feed]

In [82]:
#all_Y = [maping(session["feedback_type"]) for session in alldat]

In [84]:
all_Y[0].shape

(214,)

In [ ]:
# import torch

# # Load pre-trained weights for a model from PyTorch Hub
# model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)

In [ ]:
#from torch.utils.data import TensorDataset

In [ ]:
#----------------------------------------

In [58]:
# Define your custom Dataset class
class SpikeDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      sample = {
          "data" : torch.Tensor(self.data[idx]),
          "label" : torch.Tensor(self.labels[idx])
      }
        # data = self.data[idx],
        # label = self.labels[idx]
      return sample

In [ ]:
len(all_Y[0])

214

In [85]:
# Convert the list of numpy arrays to PyTorch tensors
#.CharTensor
#.to(torch.CharTensor)
#data = [torch.from_numpy(session).cuda().to(torch.float32) for session in all_spike_data]
data = torch.from_numpy(spikes).cuda().to(torch.float32)
# Change the data type of each tensor in the list
# for i in range(len(data)):
#     #print(data[i].dtype)
#     #data[i] = data[i].to(cuda0)
#     data[i] = data[i].to(torch.float32)
labels = [torch.tensor(labels).cuda().to(torch.float32) for labels in all_Y]
# for i in range(len(labels)):
#     #print(labels[i].dtype)
#     #labels[i] = labels[i].to(cuda0)
#     labels[i] = labels[i].to(torch.float32)

In [ ]:
for da in data:
  print(da.shape)


In [95]:
print(len(labels[15]))

280


In [98]:
labelsX = labels[0]
for lebel in labels:
  if (len(lebel) != len(labels[15])) and (len(lebel) != len(labels[0])):
    labelsX = torch.cat((labelsX, lebel), dim=0)

In [99]:
labelsX.shape

torch.Size([9770])

In [76]:
data.shape

torch.Size([9770, 130000])

In [100]:
spike_dataset = SpikeDataset(data, labelsX) # session 3

In [101]:
len(spike_dataset[0]["data"])

130000

In [102]:
len(spike_dataset[0])

2

In [123]:
class SimpleNN(nn.Module):
    def __init__(self,input_size,hid_size,out_size=1):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hid_size)
        self.fcM1 = nn.Linear(hid_size, 550)
        self.fcM2 = nn.Linear(550, 560)
        self.fcM3 = nn.Linear(560, 570)
        self.fcM4 = nn.Linear(570, 5100)
        self.fcM5 = nn.Linear(5100, 5200)
        self.fcM6 = nn.Linear(5200, 5250)
        self.fcM7 = nn.Linear(5250, 5300)
        self.fcM8 = nn.Linear(5300, 5400)
        self.fcM9 = nn.Linear(5400, 5450)
        self.fcM10 = nn.Linear(5450, 5500)
        self.fc2 = nn.Linear(5500, out_size)  # Adjust the output size based on your task
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fcM1(x))
        x = torch.relu(self.fcM2(x))
        x = torch.relu(self.fcM3(x))
        x = torch.relu(self.fcM4(x))
        x = torch.relu(self.fcM5(x))
        x = torch.relu(self.fcM6(x))
        x = torch.relu(self.fcM7(x))
        x = torch.relu(self.fcM8(x))
        x = torch.relu(self.fcM9(x))
        x = torch.relu(self.fcM10(x))
        x = torch.sigmoid(self.fc2(x))
        #x = x.view(x.size(0), -1)  # Flatten the tensor
        #x = self.fc2(x)
        return x

In [ ]:
len(spike_dataset)

214

In [103]:
# Assuming you have a dataset named 'my_dataset'
train_ratio = 0.8
train_size = int(train_ratio * len(spike_dataset))
test_size = len(spike_dataset) - train_size
train_dataset, test_dataset = random_split(spike_dataset, [train_size, test_size])

In [132]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [108]:
# Get the first batch of data
dataX = next(iter(train_loader))

# Print the batch
print(len(dataX["label"]))

16


In [109]:
len(spike_dataset[0]["data"])

130000

In [110]:
device = torch.device("cuda")
#model = model.to(device)

In [152]:
model = SimpleNN(len(spike_dataset[0]["data"]),64).to(device)
# Define your loss function
criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()
# Define your optimizer
#optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.000001,betas=(0.2,0.222))

In [141]:
summary(model, (32,len(spike_dataset[0]["data"])))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 32, 64]       8,320,064
            Linear-2              [-1, 32, 550]          35,750
            Linear-3              [-1, 32, 560]         308,560
            Linear-4              [-1, 32, 570]         319,770
            Linear-5             [-1, 32, 5100]       2,912,100
            Linear-6             [-1, 32, 5200]      26,525,200
            Linear-7             [-1, 32, 5250]      27,305,250
            Linear-8             [-1, 32, 5300]      27,830,300
            Linear-9             [-1, 32, 5400]      28,625,400
           Linear-10             [-1, 32, 5450]      29,435,450
           Linear-11             [-1, 32, 5500]      29,980,500
           Linear-12                [-1, 32, 1]           5,501
Total params: 181,603,845
Trainable params: 181,603,845
Non-trainable params: 0
-----------------------

# **Accuracy: 0.6821903787103377
# F1 Score: 0.8110739275935503**

In [153]:
# Train your model
#train_acc = []

#for indx in range(len(alldat)):
  #print("heloo")
  #train_loader = DataLoader(spike_dataset[indx], batch_size=32, shuffle=False)
  # if indx >0:
  #   model.fc1(len(spike_dataset[indx]["data"]))
  #   spike_dataset = SpikeDataset(data[indx], labels[indx])
  #   # Create the train_loader
  #   train_loader = DataLoader(spike_dataset, batch_size=32, shuffle=False)
  #for i in range(len(spike_dataset[indx][0][0])):
for epoch in range(10):
  for data in train_loader:
    #for i in range(len(spike_dataset[indx][0][0][0])):
    inputs = data["data"]
    labels = data["label"].cuda()
    #labels = spike_dataset[indx][1][i]
    optimizer.zero_grad()
    #print(inputs.dtype,inputs.size())
    #print(labels.dtype,labels.size())
    outputs = model(inputs).cuda()
    #print(outputs.dtype,outputs.size())
    loss = criterion(outputs, labels)
    print(loss)
    loss.backward()
    optimizer.step()

    # Compute accuracy
    # predicted = torch.round(torch.sigmoid(outputs))
    # correct = (predicted == labels).sum().item()
    # accuracy = correct / labels.size(0)
    # train_acc.append(accuracy)


tensor(0.2522, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2507, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2518, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2506, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2516, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2510, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2505, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2519, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2511, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2514, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2514, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2516, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2510, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2507, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2508, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2502, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.2506, device='cuda:0', grad_fn=

KeyboardInterrupt: ignored

In [154]:
from sklearn import metrics
# Set the model to evaluation mode
model.eval()

# Lists to store the predicted and true labels
predicted_labels = []
true_labels = []

# Iterate over the test_loader
for batch in test_loader:
    # Forward pass
    outputs = model(batch["data"])
    predicted = torch.round(torch.sigmoid(outputs))
    labels = batch["label"]
    # Convert tensors to numpy arrays
    predicted = predicted.cpu().detach().numpy()
    labels = labels.cpu().numpy()

    # Append the predicted and true labels to the lists
    predicted_labels.extend(predicted)
    true_labels.extend(labels)

# Convert the lists to numpy arrays
predicted_labels = np.array(predicted_labels)
true_labels = np.array(true_labels)

# Calculate accuracy and F1 score
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)
print(metrics.classification_report(true_labels, predicted_labels))
print('Accuracy:', accuracy)
print('F1 Score:', f1)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       621
         1.0       0.68      1.00      0.81      1333

    accuracy                           0.68      1954
   macro avg       0.34      0.50      0.41      1954
weighted avg       0.47      0.68      0.55      1954

Accuracy: 0.6821903787103377
F1 Score: 0.8110739275935503


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# --------------------------

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.81      0.76      0.79        17
           1       0.88      0.91      0.90        33

    accuracy                           0.86        50
   macro avg       0.85      0.84      0.84        50
weighted avg       0.86      0.86      0.86        50

[[13  4]
 [ 3 30]]


In [ ]:
# session 3 feedback accuracy
sum(y_test==prediction)/len(y_test)

0.86

In [ ]:
Yc = dat['response']

In [ ]:
Ymod = []
for i in Yc:
    if i == 0:
        x=0
    elif i == 1:
        x=1
    elif i == -1:
        x=2
    Ymod.append(x)

In [ ]:
np.unique(Ymod)

array([0, 1, 2])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(spikes_all, Ymod, test_size = 0.2, random_state = 0)

In [ ]:
import pandas as pd
from xgboost import XGBClassifier
clf=XGBClassifier(n_estimators=100, objective='multi:softmax', booster='gbtree')

In [ ]:
XGB=clf.fit(X_train,y_train)
prediction=XGB.predict(X_test)

In [ ]:
# session 3 choice accuracy
sum(y_test==prediction)/len(y_test)

0.72

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print(metrics.confusion_matrix(y_test, prediction))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        24
           1       0.38      0.30      0.33        10
           2       0.60      0.56      0.58        16

    accuracy                           0.72        50
   macro avg       0.62      0.62      0.62        50
weighted avg       0.69      0.72      0.70        50

[[24  0  0]
 [ 1  3  6]
 [ 2  5  9]]
